In [25]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np

import osu.dataset as dataset


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
# reload from saved

xs = np.load(f'.datasets/xs_5000_07-21_23-28-07.npy')
ys = np.load(f'.datasets/ys_5000_07-21_23-28-07.npy')

# xs = np.load(f'.datasets/xs_142_07-21_21-50-55.npy')
# ys = np.load(f'.datasets/ys_142_07-21_21-50-55.npy')

In [35]:
import torch

BATCH_SIZE = 128

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [56]:
import torch
import gc

# Clear GPU cache
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

1440

In [54]:
from osu.vae import OsuReplayVAE

vae = OsuReplayVAE(batch_size=BATCH_SIZE)

# vae = OsuReplayVAE.load(".trained/vae_most_recent.pt")

vae.load_data(xs, ys)

VAE models compiled..
VAE Models initialized on cuda
Encoder parameters: 236864
Decoder parameters: 260418
Beta: 1
Data loaded: 14082 training samples, 3521 test samples (position only)


In [57]:

VAE_EPOCHS = 3

for i in range(25):
    vae.train(VAE_EPOCHS)
    vae.save()
    
vae.plot_losses()

Epoch 1/3 (VAE): 100%|██████████| 111/111 [00:44<00:00,  2.50it/s]


Epoch 1/3, Total: 0.0325, Recon: 0.0192, KL: 0.0134


Epoch 2/3 (VAE): 100%|██████████| 111/111 [02:34<00:00,  1.39s/it]


Epoch 2/3, Total: 0.0068, Recon: 0.0068, KL: 0.0000


Epoch 3/3 (VAE):  32%|███▏      | 36/111 [03:47<07:53,  6.32s/it]


KeyboardInterrupt: 

In [ ]:
for i in range(25):
    vae.train(VAE_EPOCHS)
    vae.save()
    
vae.plot_losses()

In [48]:
# test training results
from osu.rulesets.mods import Mods
import osu.rulesets.beatmap as bm
import osu.dataset as dataset

test_name = 'ringyou'
test_mods = Mods.DOUBLE_TIME
test_map_path = f'assets/{test_name}_map.osu'
test_song = f'assets/{test_name}_song.mp3'

test_map = bm.load(test_map_path)
test_map.apply_mods(test_mods)

data = dataset.input_data(test_map)
data = np.reshape(data.values, (-1, dataset.BATCH_LENGTH, len(dataset.INPUT_FEATURES)))
data = torch.FloatTensor(data)

Turning beatmaps into time series data: 100%|██████████| 1/1 [00:00<00:00,  5.52it/s]


In [49]:
replay_data = vae.generate(data)
    
replay_data = np.concatenate(replay_data)
replay_data = np.pad(replay_data, ((0, 0), (0, 2)), mode='constant', constant_values=0)
if not os.path.exists('.generated'):
    os.makedirs('.generated')
    
print(f"Generated replay data shape: {replay_data.shape}")


Generated replay data shape: (8192, 4)


In [50]:
import osu.preview.preview as preview

preview.preview_replay_raw(replay_data, test_map_path, test_mods, test_song)